In [1]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from pathlib import Path

import pandas as pd

In [2]:
import sys
sys.path.append('/Users/przivic/prog/machine_learning_practico')

In [3]:
from lib import data, transformers
from lib.model import get_features_pipe, get_model_pipe

In [4]:
PATH = Path('../../data/')
movies_df = data.load_data(PATH)

Loading title basics...


/Users/przivic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Loading title ratings...
Loading movie directors...
Merging everything...


In [5]:
principals_df = pd.read_csv(PATH / 'title.principals.tsv', sep='\t')

In [6]:
principals_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [7]:
principals_df.category.value_counts()

actor                  9428011
actress                7087536
self                   6992466
writer                 5443248
director               4739306
producer               2533985
cinematographer        1472541
composer               1461940
editor                 1375324
production_designer     302528
archive_footage         245691
archive_sound             2605
Name: category, dtype: int64

In [8]:
# Sacado del codigo de directores

movies_stars = principals_df[principals_df.category.isin(['actress', 'actor'])].copy()

# Calculo un ranking por pelicula segun el ordering
movies_stars['star_rank'] = (
    movies_stars.sort_values('ordering')
        .groupby('tconst')
        .cumcount()
)

# Me quedo con el "director principal" por pelicula
first_star = movies_stars[movies_stars.star_rank == 0][['nconst', 'tconst']].rename(columns={'nconst': '1st_star'})
second_star = movies_stars[movies_stars.star_rank == 1][['nconst', 'tconst']].rename(columns={'nconst': '2nd_star'})
third_star = movies_stars[movies_stars.star_rank == 2][['nconst', 'tconst']].rename(columns={'nconst': '3rd_star'})

In [9]:
stars_df = first_star.merge(second_star, how='left', on='tconst').merge(third_star, how='left', on='tconst')

In [10]:
stars_df.head()

,1st_star,tconst,2nd_star,3rd_star
0,nm0443482,tt0000005,nm0653042,NaN
1,nm0179163,tt0000007,nm0183947,NaN
2,nm0653028,tt0000008,NaN,NaN
3,nm0063086,tt0000009,nm0183823,nm1309758
4,nm3692297,tt0000011,NaN,NaN


In [11]:
stars_df[stars_df.tconst == 'tt0120338']

,1st_star,tconst,2nd_star,3rd_star
107363,nm0000138,tt0120338,nm0000701,nm0000708


In [12]:
movies_df = movies_df.merge(stars_df, on='tconst', how='left')

In [13]:
movies_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,director,1st_star,2nd_star,3rd_star
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894.0,\N,45.0,[Romance],5.9,154,nm0085156,nm0063086,nm0183823,nm1309758
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897.0,\N,20.0,"[Documentary, News, Sport]",5.2,356,nm0714557,NaN,NaN,NaN
2,tt0000502,movie,Bohemios,Bohemios,0,1905.0,\N,100.0,[no-genre],3.8,6,nm0063413,nm0215752,nm0252720,NaN
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906.0,\N,70.0,"[Biography, Crime, Drama]",6.1,589,nm0846879,nm0846887,nm0846894,nm3002376
4,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908.0,\N,120.0,"[Adventure, Fantasy]",5.2,37,nm0091767,nm0000875,nm0122665,nm0933446


# Ahora  vamos a experimentar!

Como podemos hacer para usar 1st_star y 2nd_star con el código que **ya** tenemos? [Miremos el diff](https://github.com/elsonidoq/machine_learning_practico/commit/1244da3daee2f7aff140d202885e6e8dba55c099)

In [14]:
rating_data = data.load_rating_train_dev_test(movies_df)

In [15]:
pipe = make_pipeline(
    transformers.CrewFeatures('1st_star', min_cnt_movies=3),
    DictVectorizer(sparse=False),
    StandardScaler(),
    LogisticRegression()
)

In [16]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.6694917991958195, 0.5516081278474014)

In [19]:
pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.CrewFeatures('1st_star', min_cnt_movies=5), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('2nd_star', min_cnt_movies=5), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('3rd_star', min_cnt_movies=5), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression()
)

In [20]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

(0.6891875295967093, 0.5639912339014951)

# Probando todo junto

In [25]:
pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.CrewFeatures('1st_star', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('2nd_star', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('3rd_star', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('director', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(transformers.GenreDummies(), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression()
)

In [ ]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc